## Functions

In [648]:
import LyndonWords
import importlib
importlib.reload(LyndonWords)
from LyndonWords import *
import itertools
from functools import partial
import multiprocessing as mp
from tqdm.notebook import tqdm
import math
import numpy as np
def gen_permutations(type,n):
    for perm in itertools.permutations(np.array(range(n+1))):
        yield rootSystem(perm,type)

### Monotonicity Stuff

In [649]:
def monotonicity_conj(rootsys:rootSystem,wordToCheck:word):
    wordPlusDelta = rootsys.get_words(wordToCheck.weights + wordToCheck.delta)[0]
    parsedFormat = rootsys.parse_to_delta_format(wordPlusDelta)
    if(type(parsedFormat[0]) is list):
        return True
    return False
#TODO: Updated bases on new claim on monotonicity
def check_monotone_increasing_imp_cond(rootSystemPermutations):
    rootsys:rootSystem
    for rootsys in rootSystemPermutations:
        returnArr = []
        deltaWords = rootsys.get_words(rootsys.delta)
        temp = rootsys.check_monotonicity("Increasing")
        monotoneIncreasing = [rootsys.get_words(i[0] + rootsys.delta)[0] for i in temp]
        for i in monotoneIncreasing:
            flag = False
            for j in deltaWords:
                if(not str(i).startswith(str(j)) and (i.weights - rootsys.delta)[rootsys.ordering[0].rootIndex] > 0):
                    flag = True
                    break
            if(flag):
                continue
            returnArr.append((str(rootsys.ordering),(i.weights - rootsys.delta)))
    return returnArr

In [650]:
def check_cond_imp_monotone_increasing(rootSystemPermutations):
    returnArr = []
    rootsys:rootSystem
    for rootsys in rootSystemPermutations:
        deltaWords = rootsys.get_words(rootsys.delta)
        deltaArrays = [i.string for i in deltaWords]
        for base in rootsys.baseWeights[:-1]:
            secondWord = rootsys.get_words(base + rootsys.delta)[0]
            if(base[rootsys.ordering[0].rootIndex] == 0):
                continue
            deltaStart = False
            for i in deltaArrays:
                if(np.all(secondWord[:rootsys.deltaHeight] == i)):
                    deltaStart = True
                    break
            if deltaStart:
                continue
            if(rootsys.get_monotonicity(base) != 1):
                returnArr.append((str(rootsys.ordering),str(rootsys.get_words(base)[0])))
    return returnArr

### Printing and Exporting

In [651]:
def print_by_length(rootSystem:rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.get_words(comb):
                print(" "+str(i))

In [652]:
def print_by_base_word(rootSystem:rootSystem,comb):
    k=0
    is_imaginary = rootSystem.is_imaginary(comb)
    for i in rootSystem.get_affine_words(comb):
        if(is_imaginary and k % rootSystem.n == 0):
            print(k //rootSystem.n)
        if(not is_imaginary):
            print(k)
        print(" " + str(i))
        k+=1

In [653]:
def export_by_length(rootsys:rootSystem,quoteMarks=False):
    file = open("out.txt",'w')
    file.write(str(rootsys.ordering)+"\n")
    for k in range(rootsys.k + 1):
        for i in rootsys.baseWeights:
            comb = i + rootsys.delta * k
            for i in rootsys.get_words(comb):
                if(quoteMarks):
                    file.write(f"'{str(i)}',")
                else:
                    file.write(str(i) + '\n')
    file.close()

### Delta Pattern Detection

In [654]:
def delta_split_at_last(rootsys:rootSystem,index: int) -> bool:
    baseWord = rootsys.get_words(rootsys.delta)[index]
    deltaWords = rootsys.get_affine_words(rootsys.delta)[index + rootsys.n::rootsys.n]
    for deltaWord in deltaWords:
        parsed = rootsys.parse_to_delta_format(deltaWord)
        if(len(parsed) != 3):
            return False
        if(parsed[2] != str(baseWord[-1])):
            return False
    return True

def delta_split_at_cofac(rootsys: rootSystem,index: int) -> bool:
    baseWord = rootsys.get_words(rootsys.delta)[index]
    cofacSplit = rootsys.costfac(baseWord)[0].height
    for deltaWord in rootsys.get_affine_words(rootsys.delta)[index+rootsys.n::rootsys.n]:
        parsed = rootsys.parse_to_delta_format(deltaWord)
        if(len(parsed) != 3):
            return False
        if(parsed[0] != baseWord.noCommas()[:cofacSplit] or parsed[2] != baseWord.noCommas()[cofacSplit:]):
            return False
    return True

def k3_start_delta_pattern(rootsys: rootSystem, index: int) ->bool:
    deltaWords = rootsys.get_affine_words(rootsys.delta)[index::rootsys.n]
    splitting = [i.noCommas() for i in rootsys.costfac(deltaWords[1])]
    if(len(rootsys.parse_to_delta_format(deltaWords[1])) != 1):
        return False
    for i in deltaWords[2:]:
        parsed = rootsys.parse_to_delta_format(i)
        if(len(parsed) != 3):
            return False
        if(parsed[0] != splitting[0] or parsed[2] != splitting[1] or type(parsed[1]) is not list):
            return False
    return True

def last_smallest_delta_pattern(rootsys:rootSystem, index: int) -> bool:
    deltaWords = rootsys.get_affine_words(rootsys.delta)[index::rootsys.n]
    for i in range(-1,-rootsys.deltaHeight-1,-1):
        if(deltaWords[0][i].rootIndex == rootsys.ordering[0].rootIndex):
            smallestIndex = rootsys.deltaHeight + i
            break
    for deltaWord in deltaWords[2:]:
        parsed = rootsys.parse_to_delta_format(deltaWord)
        if(len(parsed) != 3):
            return False
        if(smallestIndex != len(parsed[0])):
            return False
    return True

def two_delta_words_delta_pattern(rootsys:rootSystem,index:int) -> bool:
    deltaWords = rootsys.get_affine_words(rootsys.delta)[index+rootsys.n*2::rootsys.n]
    for deltaWord in deltaWords:
        parsed = rootsys.parse_to_delta_format(deltaWord)
        if(len(parsed) != 4):
            return False
        if(type(parsed[0]) != str or type(parsed[1]) != list or type(parsed[2]) != list or type(parsed[3])!= str):
            return False
    return True

In [655]:
class deltaTypes:
    def __init__(self,index:int,hs:list,type:str,insertedIndex:int,leftfac:str,rightfac:str):
        self.index = index
        self.hs = hs
        #is either "last" or "cofac" or "neither" if something goes horribly wrong
        self.type = type
        self.insertedIndex = insertedIndex
        self.leftfac = leftfac
        self.rightfac = rightfac
    def to_list(self):
        return [self.index,self.type,self.insertedIndex,self.leftfac,self.rightfac]
class deltaTypesCollection:
    def __init__(self,rootsys:rootSystem,deltaTypesList):
        self.type = rootsys.type
        self.ordering = str(rootsys.ordering)
        self.deltaTypes = deltaTypesList
    def to_csv(self)->str:
        retstr = f"{self.type},{self.ordering},,,,\n"
        for i in self.deltaTypes:
            retstr+= f"{i.index},[{' '.join([str(j) for j in i.hs])}],{i.type},{i.insertedIndex},{i.leftfac},{i.rightfac}\n"
        return retstr
    def not_all_last(self) -> bool:
        for i in self.deltaTypes:
            if i.type != "last":
                return True
        return False

In [656]:
def generate_delta_types(rootsys:rootSystem,k=3) ->deltaTypesCollection:
    deltaWords = rootsys.get_words(rootsys.delta)
    listOfDeltaTypes = []
    kdeltaWords = rootsys.get_words(rootsys.delta*k)
    for i in range(rootsys.n):
        kdeltaWord = kdeltaWords[i]
        splitting = 0
        parsedForm = rootsys.parse_to_delta_format(kdeltaWord)
        for j in parsedForm:
            if(type(j) is not str):
                splitting = j[0]
                break
        if(delta_split_at_last(rootsys,i)):
            breakType = "last"
        elif(delta_split_at_cofac(rootsys,i)):
            breakType = "cofac"
        elif(k3_start_delta_pattern(rootsys,i)):
            breakType = "k3start"
        elif(last_smallest_delta_pattern(rootsys,i)):
            breakType = "lastSmallest"
        elif(two_delta_words_delta_pattern(rootsys,i)):
            breakType = "twoDeltaWords"
        else:
            breakType = "other"
        factors = rootsys.costfac(deltaWords[i])
        listOfDeltaTypes.append(deltaTypes(i+1,deltaWords[i].hs,breakType,splitting,factors[0].noCommas(),factors[1].noCommas()))
    return deltaTypesCollection(rootsys,listOfDeltaTypes)
        

In [657]:
def costfac_delta_type_conditions_met(rootsys:rootSystem,deltaWords,index):
    factorization = rootsys.costfac(deltaWords[index])
    for i in range(index):
        if(factorization[1] < deltaWords[i]):
            if(rootsys.eBracket(rootsys.costfac(deltaWords[index])[1] + deltaWords[i])):
                return True
        else:
            break
    minusarr = np.zeros(rootsys.n+1,dtype=int)
    minusarr[deltaWords[index][-1].rootIndex] += 1
    if(word.letter_list_cmp(rootsys.get_words(rootsys.delta - minusarr)[0].string,deltaWords[index][:-1]) != 0):
        return True
    for j in range(index):
        if(word.letter_list_cmp(rootsys.costfac(deltaWords[j])[0].string,
                        rootsys.costfac(deltaWords[index])[1].string[:-1]) == 0):
            return True
    return False
def last_split_delta_type_conditions_met(rootsys:rootSystem,deltaWords:list,index:int) -> bool:
    if(rootsys.delta[rootsys.ordering[0].rootIndex] == 1):
        return True
    rightfac = rootsys.costfac(deltaWords[index])[1]
    for deltaWord in deltaWords[:index]:
        if(word.letter_list_cmp(rootsys.costfac(deltaWord)[1].string,rightfac.string[:-1]) == 0):
            return True
    return False
def k3_start_delta_type_conditions_met(rootsys:rootSystem,index:int) -> bool:
    if(index == 0):
        return False
    deltaWords = rootsys.get_words(rootsys.delta)
    endings = np.array([rootsys.costfac(i)[1].weights for i in deltaWords[:index]])
    indexWeight = np.array(rootsys.costfac(deltaWords[index])[1].weights)
    for end in endings:
        if(sum(indexWeight)-1 == sum(end) and sum(indexWeight - end) == 1):
            return False
    for i in range(len(endings)):
        if(endings[i][-1] == 1):
            endings[i] = rootsys.delta - endings[i]
    indexWeight[deltaWords[index][-1].rootIndex] -= 1
    if(indexWeight[-1] == 1):
        indexWeight = rootsys.delta - indexWeight
    np.append(endings,[indexWeight],axis=0)
    return np.linalg.matrix_rank(endings) == index
def check_delta_type_prediction(rootsys:rootSystem,k=2):
    result = generate_delta_types(rootsys,k)
    deltaWords = rootsys.get_words(rootsys.delta)
    for i in range(len(result.deltaTypes)):
        if((result.deltaTypes[i].type == "cofac") != costfac_delta_type_conditions_met(rootsys,deltaWords,i)):
            yield (rootsys.type, str(rootsys.ordering),i+1,result.deltaTypes[i].type,"cofac")
        #if((result.deltaTypes[i].type == "last") != lastSplitDeltaTypeConditionsMet(rootsys,deltaWords,i)):
        #    exceptions.append((rootsys.type, str(rootsys.ordering),i+1,result.deltaTypes[i].type,"last"))
        #if((result.deltaTypes[i].type == "k3start") != k3StartDeltaTypeConditionsMet(rootsys,i)):
        #   yield (rootsys.type, str(rootsys.ordering),i+1,result.deltaTypes[i].type,"k3start") 
def check_delta_type_prediction_perms(rootsystems,k=2):
    rootsys:rootSystem
    for rootsys in rootsystems:
        for i in check_delta_type_prediction(rootsys,k):
            yield i

In [658]:
def write_delta_types_to_file(rootSystems,file="out.txt",notOnlyAllLast=True):
    fileobj = open(file,'w')
    fileobj.write("imaginary index, bracketing, splitting type, inserted imaginary index, left factor, right factor\n")
    i:rootSystem
    for i in rootSystems:
        deltaTypesCol:deltaTypesCollection = generate_delta_types(i)
        if(notOnlyAllLast):
            if(not deltaTypesCol.not_all_last()):
                continue
        fileobj.write(deltaTypesCol.to_csv())
    fileobj.close()

### SL Word Algorithm

In [659]:
def check_SL_word_algo_perms(rootsyss,k,equal=False):
    for rootsys in rootsyss:
        for exc in check_SL_word_algo(rootsys,k,equal):
            yield (str(rootsys.ordering),exc,rootsys.parse_to_delta_format(rootsys.get_words(exc)[0]),rootsys.parse_to_delta_format(rootsys.SL_word_algo(exc)))
def check_SL_word_algo(rootsys:rootSystem,k,equal=False):
    for kprime in range(k):
        for baseWeight in rootsys.baseWeights[:-1]:
            weight = baseWeight + kprime*rootsys.delta
            if(equal):
                if(word.letter_list_cmp(rootsys.get_words(weight)[0].string,rootsys.SL_word_algo(weight).string) != 0):
                    yield weight
            else:
                if(word.letter_list_cmp(rootsys.get_words(weight)[0].string,rootsys.SL_word_algo(weight).string) > 0):
                    yield weight 

### Imaginary Words Stuff

In [660]:
def check_h_deltas_repeat(perms):
    excepts = []
    i:rootSystem
    for i in perms:
        hs = [j.hs for j in i.get_affine_words(i.delta)]
        bases = hs[i.n:2*i.n]
        for offset in range(0,i.n):
            for k in range(2,len(hs)//i.n):
                if(np.any(bases[offset] != hs[offset + k*i.n])):
                    excepts.append((str(i.ordering),f"SL_{offset+1}({k}\delta)"))
    return excepts

In [661]:
def check_one_smallest_last(perms):
    rootsys:rootSystem
    for rootsys in perms:
        if(rootsys.delta[rootsys.ordering[0].rootIndex] == 1):
            if(rootsys.get_words(rootsys.delta)[0][-1].rootIndex != rootsys.ordering[1].rootIndex):
                yield (str(rootsys.ordering),rootsys.get_words(rootsys.delta)[0].noCommas(),rootsys.ordering[1].rootIndex)

### Rootsystem equivalences

In [662]:
def compare_rootsystems(r1,r2):
    words = list(r1.weightToWordDictionary.values())
    for i in words:
        for j in range(len(i)):
            if(str(i[j]) != str(r2.get_words(i[j].weights)[j])):
                yield (i[j].weights,str(i[j]),str(r2.get_words(i[j].weights)[j]))

In [663]:
def generate_equivalences(permutations):
    equivalences = np.arange(len(permutations))
    for i in range(len(permutations)):
        for j in range(0,i):
            if(equivalences[j] != j):
                continue
            flag = False
            for k in compare_rootsystems(permutations[i],permutations[j]):
                flag =True
                break
            if(not flag):
                yield (permutations[i].ordering,permutations[j].ordering)

### Periodicity

In [664]:
def get_max_periodicity(rootSystems):
    max = 0
    maxOrdering = ""
    maxRoot = []
    rootsys:rootSystem
    for rootsys in rootSystems:
        for simpleRoot in rootsys.baseWeights[:-1]:
            result = rootsys.get_periodicity(simpleRoot)
            if(result > max):
                max = result
                maxOrdering = str(rootsys.ordering)
                maxRoot=simpleRoot
    return (max,maxOrdering,maxRoot)

In [665]:
def periodicity_conj(rootsys:rootSystem,deltas, weights) -> int:
    if(weights[rootsys.ordering[0].rootIndex] != 0):
        return 1
    for d in deltas:
        hs = rootsys.cartan_matrix @ d.hs
        f = False   
        for v in range(len(weights)):
            if(v == 0 and weights[v] > 0):
                if(np.dot(rootsys.delta[1:],hs) != 0):
                    f=True
                    break
            else:
                if(weights[v] > 0 and hs[v-1] != 0):
                    f=True
                    break
        if(f):
            break
    s = 0
    for v in range(len(weights)):
            if(v == 0 and weights[v] > 0):
                if(np.dot(rootsys.delta[1:],hs) != 0):
                    s+=1
            else:
                if(weights[v] > 0 and hs[v-1] != 0):
                    s+=weights[v]
    return s

In [666]:
def periodicity_conj_root(rootsys:rootSystem, weights) :
    return periodicity_conj(rootsys,rootsys.get_words(rootsys.delta),weights)

In [667]:
def check_periodicity_conj_rootsystem(rootsys:rootSystem):
    if(rootsys.delta[rootsys.ordering[0].rootIndex] == 1):
        for l in rootsys.baseWeights[:-1]:
            if(l[rootsys.ordering[0].rootIndex] == 0):
                period = rootsys.get_periodicity(l)
                conjperiod = periodicity_conj_root(rootsys,l)
                if(period != conjperiod):
                    yield (str(rootsys.ordering),l,period,conjperiod)

In [668]:
def check_periodicity_conj_perms(perms):
    for p in perms:
        yield check_periodicity_conj_rootsystem(p)

## Type A

In [572]:
Aperms = gen_permutations('A',4)

In [333]:
periodicity_conj_root(rootSystem([0,1,2,3,4],'A'),[0,0,1,0,0])

1

In [407]:
for i in check_periodicity_conj_perms(Aperms):
    print(i)

<generator object check_periodicity_conj_rootsystem at 0x0000029A501B06D0>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4FC1B2E0>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4FC1BD30>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4FC19E40>
<generator object check_periodicity_conj_rootsystem at 0x0000029A501B06D0>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4FC19E40>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4F203790>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4FC19E40>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4F203790>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4FC19E40>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4F203790>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4FC19E40>
<generator object check_periodicity_conj_rootsystem at 0x0000029A4F203C40>
<generator object check_p

In [571]:
A5 = rootSystem([0,1,2,3,4,5],'A')
w = [3, 3, 2, 3, 3, 3]
print(A5.get_words(w)[0])
print(A5.SL_word_algo(w))

0,5,4,3,1,0,5,4,3,2,1,0,5,4,3,2,1
0,5,4,3,1,0,5,4,3,2,1,0,5,4,3,2,1


In [573]:
for i in check_SL_word_algo_perms(Aperms,3,True):
    print(i)

('0<1<2<3<4<5', array([1, 1, 1, 2, 1, 1]), [[2, 1], '3'], [[1, 1]])
('0<1<2<3<4<5', array([1, 1, 1, 1, 2, 1]), [[3, 1], '4'], [[1, 1]])
('0<1<2<3<4<5', array([1, 1, 1, 1, 1, 2]), [[4, 1], '5'], [[1, 1]])
('0<1<2<3<4<5', array([1, 1, 1, 2, 2, 1]), [[2, 1], '34'], [[1, 1]])
('0<1<2<3<4<5', array([1, 1, 1, 1, 2, 2]), [[3, 1], '45'], [[1, 1]])
('0<1<2<3<4<5', array([1, 1, 1, 2, 2, 2]), [[2, 1], '345'], [[1, 1]])
('0<1<2<3<4<5', array([2, 2, 2, 1, 2, 2]), ['05412', [2, 1]], ['05412', [1, 1]])
('0<1<2<3<4<5', array([2, 2, 2, 2, 1, 2]), ['05123', [3, 1]], ['05123', [1, 1]])
('0<1<2<3<4<5', array([2, 2, 2, 2, 2, 1]), ['01234', [4, 1]], ['01234', [1, 1]])
('0<1<2<3<4<5', array([2, 2, 2, 3, 2, 2]), [[2, 2], '3'], [[1, 1]])
('0<1<2<3<4<5', array([2, 2, 2, 2, 3, 2]), [[3, 2], '4'], [[1, 1]])
('0<1<2<3<4<5', array([2, 2, 2, 2, 2, 3]), [[4, 2], '5'], [[1, 1]])
('0<1<2<3<4<5', array([2, 2, 2, 3, 3, 2]), [[2, 2], '34'], [[1, 1]])
('0<1<2<3<4<5', array([2, 2, 2, 2, 3, 3]), [[3, 2], '45'], [[1, 1]])
('0

KeyboardInterrupt: 

In [337]:
for i in check_delta_type_prediction_perms(Aperms):
    print(i)

In [338]:
write_delta_types_to_file(Aperms,"ADeltaTypes.csv")

In [339]:
get_max_periodicity(Aperms)

(0, '', [])

## Type C

In [185]:
Cperms = gen_permutations('C',5)

In [119]:
for i in check_periodicity_conj_perms(Cperms):
    print(i)

('0<2<1<3<4<5', array([1, 0, 0, 0, 0, 0]), 2, 1)
('0<2<1<3<4<5', array([0, 2, 2, 2, 2, 1]), 3, 6)
('0<2<1<3<5<4', array([1, 0, 0, 0, 0, 0]), 2, 1)
('0<2<1<3<5<4', array([0, 2, 2, 2, 2, 1]), 3, 6)
('0<2<1<4<3<5', array([1, 0, 0, 0, 0, 0]), 2, 1)
('0<2<1<4<3<5', array([0, 2, 2, 2, 2, 1]), 3, 6)
('0<2<1<4<5<3', array([1, 0, 0, 0, 0, 0]), 2, 1)
('0<2<1<4<5<3', array([0, 2, 2, 2, 2, 1]), 3, 6)
('0<2<1<5<3<4', array([1, 0, 0, 0, 0, 0]), 2, 1)
('0<2<1<5<3<4', array([0, 2, 2, 2, 2, 1]), 3, 6)
('0<2<1<5<4<3', array([1, 0, 0, 0, 0, 0]), 2, 1)
('0<2<1<5<4<3', array([0, 2, 2, 2, 2, 1]), 3, 6)
('0<2<3<1<4<5', array([1, 0, 0, 0, 0, 0]), 2, 1)
('0<2<3<1<4<5', array([0, 2, 2, 2, 2, 1]), 3, 6)
('0<2<3<1<5<4', array([1, 0, 0, 0, 0, 0]), 2, 1)
('0<2<3<1<5<4', array([0, 2, 2, 2, 2, 1]), 3, 6)
('0<2<3<4<1<5', array([1, 0, 0, 0, 0, 0]), 2, 1)
('0<2<3<4<1<5', array([0, 2, 2, 2, 2, 1]), 3, 6)
('0<2<3<4<5<1', array([1, 0, 0, 0, 0, 0]), 2, 1)
('0<2<3<4<5<1', array([0, 2, 2, 2, 2, 1]), 3, 6)
('0<2<3<5<1<4', arra

In [186]:
for i in check_SL_word_algo_perms(Cperms,3,True):
    print(i)

('0<1<2<3<4<5', array([1, 2, 2, 3, 2, 1]), [[2, 1], '3'], [[1, 1]])
('0<1<2<3<4<5', array([1, 2, 2, 2, 3, 1]), [[3, 1], '4'], [[1, 1]])
('0<1<2<3<4<5', array([1, 2, 2, 2, 2, 2]), [[4, 1], '5'], [[1, 1]])
('0<1<2<3<4<5', array([1, 2, 2, 2, 3, 2]), [[3, 1], '45'], [[1, 1]])
('0<1<2<3<4<5', array([1, 2, 2, 3, 3, 1]), [[2, 1], '34'], [[1, 1]])
('0<1<2<3<4<5', array([1, 2, 2, 3, 3, 2]), [[2, 1], '345'], [[1, 1]])
('0<1<2<3<4<5', array([1, 2, 2, 2, 4, 2]), [[3, 1], '454'], [[1, 1]])
('0<1<2<3<4<5', array([1, 2, 2, 3, 4, 2]), [[2, 1], '3454'], [[1, 1]])
('0<1<2<3<4<5', array([1, 2, 2, 4, 4, 2]), [[2, 1], '34543'], [[1, 1]])
('0<1<2<3<4<5', array([2, 4, 4, 5, 4, 2]), [[2, 2], '3'], [[1, 2]])
('0<1<2<3<4<5', array([2, 4, 4, 4, 5, 2]), [[3, 2], '4'], [[1, 2]])
('0<1<2<3<4<5', array([2, 4, 4, 4, 4, 3]), [[4, 2], '5'], [[1, 2]])
('0<1<2<3<4<5', array([2, 4, 4, 4, 5, 3]), [[3, 2], '45'], [[1, 2]])
('0<1<2<3<4<5', array([2, 4, 4, 5, 5, 2]), [[2, 2], '34'], [[1, 2]])
('0<1<2<3<4<5', array([2, 4, 4, 5

KeyboardInterrupt: 

In [ ]:
for i in check_delta_type_prediction_perms(Cperms):
    print(i)

  0%|          | 0/720 [00:00<?, ?it/s]

In [ ]:
write_delta_types_to_file(Cperms,"CDeltaTypes.csv")

  0%|          | 0/720 [00:00<?, ?it/s]

In [ ]:
print(*[f"{str(i[0])}: {i[1]}\n" for i in check_monotone_increasing_imp_cond(Cperms)])

5<4<3<2<1<0: [0 0 0 0 0 1]
 5<4<3<2<1<0: [1 0 0 0 0 1]
 5<4<3<2<1<0: [1 1 0 0 0 1]
 5<4<3<2<1<0: [2 0 0 0 0 1]
 5<4<3<2<1<0: [2 1 0 0 0 1]
 5<4<3<2<1<0: [2 2 0 0 0 1]



In [ ]:
check_cond_imp_monotone_increasing(Cperms)

[('1<3<0<2<4<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<2<5<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<4<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<4<5<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<5<2<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<5<4<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<0<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<0<5<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<5<0<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<0<2<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<0<4<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<4<0<2', '1,2,0,3,4,5,4,3,2'),
 ('1<4<0<2<3<5', '1,2,3,0,4,5,4,3'),
 ('1<4<0<2<3<5', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<2<5<3', '1,2,3,0,4,5,4,3'),
 ('1<4<0<2<5<3', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<3<2<5', '1,2,3,0,4,5,4,3'),
 ('1<4<0<3<2<5', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<3<5<2', '1,2,3,0,4,5,4,3'),
 ('1<4<0<3<5<2', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<5<2<3', '1,2,3,0,4,5,4,3'),
 ('1<4<0<5<2<3', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<5<3<2', '1,2,3,0,4,5,4,3'),
 ('1<4<0<5<3<2', '1,2,3,0,4,5,4,3,2'),
 ('1<4<3<0<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<4<3<0<5<2', '1,2,0,3,4,5,4,3,2')

In [ ]:
check_h_deltas_repeat(Cperms)

[]

In [ ]:
index=250
print(Cperms[index].ordering)
[print(*list(Cperms[index].costfac(i))+[i.hs],sep='-',) for i in Cperms[index].get_affine_words(Cperms[index].delta)]

2<0<3<5<1<4
2,1,3,4,0-2,1,3,4,5-[1 1 1 1 1]
2,1,3,4-2,1,3,4,5,0-[1 1 1 1 0]
2,1,3,0-2,1,3,4,5,4-[1 1 1 2 1]
2,1,0-2,1,3,4,5,4,3-[1 1 2 2 1]
2,3,4,5,4,3-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,4-[0 0 0 1 0]
2,1,0,2,1,3,4,5,4-2,1,3,0,2,1,3,4,5,4,3-[0 0 1 0 0]
2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,4-[0 0 0 1 0]
2,1,0,2,1,3,4,5,4-2,1,3,0,2,1,3,4,5,4,2,1,3,0,2,1,3,4,5,4,3-[0 0 1 0 0]
2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
for i in generate_equivalences(Cperms):
    print(f"{i[0]} = {i[1]}")

0<1<3<4<5<2 = 0<1<3<4<2<5
0<1<4<3<5<2 = 0<1<4<3<2<5
0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<4<5<1<3 = 0<2<4<1<5<3
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<1<4<5<2 = 0<3<1<4<2<5
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<1<5<2 = 0<3<4<1<2<5
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<4<5<1<2 = 0<3<4<1<2<5
0<3<4<5<1<2 = 0<3<4<1<5<2
0<3<4<5<2<1 = 0<3<4<2<1<5
0<3<4<5<2<1 = 0<3<4<2<5<1
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<1<3<5<2 = 0<4<1<3<2<5
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<2<5<1<3 = 0<4<2<1<5<3
0<4<3<1<5<2 = 0<4<3<1<2<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<4<3<5<1<2 = 0<4<3<1<2<5
0<4<3<5<1<2 = 0<4<3<1<5<2
0<4<3<5<2<1 = 0<4<3<2<1<5
0<4<3<5<2<1 = 0<4<3<2<5<1
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<0<3<4<5<2 = 1<0<3<4<2<5
1<0<4<3<5<2 = 1<0<4<3<2<5
1<2<3<0<4<5 

In [ ]:
write_delta_types_to_file(Cperms)

In [79]:
get_max_periodicity(Cperms)

(5, '0<3<1<4<2', array([0, 0, 2, 2, 1]))

## Type G

In [574]:
G2perms = gen_permutations('G',2)

In [46]:
for i in check_periodicity_conj_perms(G2perms):
    print(i)

In [19]:
get_max_periodicity(G2perms)

(5, '1<0<2', array([1, 1, 3]))

In [575]:
for i in check_SL_word_algo_perms(G2perms,5,equal=True):
    print(i)

('2<0<1', array([1, 3, 4]), [[2, 1], '21'], [[1, 1]])
('2<0<1', array([2, 3, 5]), [[2, 1], '2210'], ['2210', [1, 1]])
('2<0<1', array([2, 5, 7]), [[2, 2], '21'], [[1, 1]])
('2<0<1', array([3, 5, 8]), [[2, 2], '2210'], ['2210', [1, 2]])
('2<0<1', array([ 3,  7, 10]), [[2, 3], '21'], [[1, 1]])
('2<0<1', array([ 4,  7, 11]), [[2, 3], '2210'], ['2210', [1, 3]])
('2<0<1', array([ 4,  9, 13]), [[2, 4], '21'], [[1, 1]])
('2<0<1', array([ 5,  9, 14]), [[2, 4], '2210'], ['2210', [1, 4]])
('2<1<0', array([1, 3, 4]), [[2, 1], '21'], [[1, 1]])
('2<1<0', array([2, 3, 5]), [[2, 1], '2210'], ['2210', [1, 1]])
('2<1<0', array([2, 5, 7]), [[2, 2], '21'], [[1, 1]])
('2<1<0', array([3, 5, 8]), [[2, 2], '2210'], ['2210', [1, 2]])
('2<1<0', array([ 3,  7, 10]), [[2, 3], '21'], [[1, 1]])
('2<1<0', array([ 4,  7, 11]), [[2, 3], '2210'], ['2210', [1, 3]])
('2<1<0', array([ 4,  9, 13]), [[2, 4], '21'], [[1, 1]])
('2<1<0', array([ 5,  9, 14]), [[2, 4], '2210'], ['2210', [1, 4]])


In [ ]:
for i in check_delta_type_prediction_perms(G2perms):
    print(i)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
write_delta_types_to_file(G2perms,"GDeltaTypes.csv",False)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
check_cond_imp_monotone_increasing(G2perms)

[('1<0<2', '1,2,2,2'), ('1<0<2', '1,2,2,2,0')]

In [ ]:
check_monotone_increasing_imp_cond(G2perms)

[]

In [ ]:
for i in G2perms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
check_h_deltas_repeat(G2perms)

[]

In [ ]:
for i in generate_equivalences(G2perms):
    print(f"{i[0]} = {i[1]}")

## Type B

In [22]:
Bperms = gen_permutations('B',6)

In [134]:
for i in check_SL_word_algo_perms(Bperms,4):
    print(i)

KeyboardInterrupt: 

In [ ]:
for i in check_SL_word_algo_perms(Bperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

ValueError: Algorithm only word if the smallest occurs once

In [ ]:
for i in check_delta_type_prediction_perms(Bperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

In [ ]:
write_delta_types_to_file(Bperms,"BDeltaTypes.csv")

  0%|          | 0/5040 [00:00<?, ?it/s]

In [ ]:
check_cond_imp_monotone_increasing(Bperms)

[('2<0<1<3<4<5', '2,3,4,5,5,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,1,0'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,3,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,1,0'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,3,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,3,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,3,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<5<3<4', '2,3,4,5,5,1'),
 ('2<0<1<5<3<4', '2,3,4,5,5,4,1'),
 ('2<0<1<5<3<4', '2,3,4,5,5,1,0'),
 ('2<0<1<5<3<4', '2,3,4,5,5,4,3,1

In [ ]:
check_monotone_increasing_imp_cond(Bperms)

[]

In [ ]:
for i in Bperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
check_h_deltas_repeat(Bperms)

[]

In [ ]:
for i in generate_equivalences(Bperms):
    print(f"{i[0]} = {i[1]}")

0<1<3<4<5<2 = 0<1<3<4<2<5
0<1<4<3<5<2 = 0<1<4<3<2<5
0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<4<5<1<3 = 0<2<4<1<5<3
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<1<4<5<2 = 0<3<1<4<2<5
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<1<5<2 = 0<3<4<1<2<5
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<4<5<1<2 = 0<3<4<1<2<5
0<3<4<5<1<2 = 0<3<4<1<5<2
0<3<4<5<2<1 = 0<3<4<2<1<5
0<3<4<5<2<1 = 0<3<4<2<5<1
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<1<3<5<2 = 0<4<1<3<2<5
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<2<5<1<3 = 0<4<2<1<5<3
0<4<3<1<5<2 = 0<4<3<1<2<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<4<3<5<1<2 = 0<4<3<1<2<5
0<4<3<5<1<2 = 0<4<3<1<5<2
0<4<3<5<2<1 = 0<4<3<2<1<5
0<4<3<5<2<1 = 0<4<3<2<5<1
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<0<3<4<5<2 = 1<0<3<4<2<5
1<0<4<3<5<2 = 1<0<4<3<2<5
1<2<3<4<0<5 

## Type D

In [24]:
Dperms = gen_permutations('D',6)

In [136]:
for i in check_SL_word_algo_perms(Dperms,3):
    print(i)

KeyboardInterrupt: 

In [ ]:
check_delta_type_prediction_perms(Dperms)

  0%|          | 0/40320 [00:00<?, ?it/s]

[]

In [ ]:
write_delta_types_to_file(Dperms,"DDeltaTypes.csv")

  0%|          | 0/5040 [00:00<?, ?it/s]

In [ ]:
check_monotone_increasing_imp_cond(Dperms)

[('5<4<3<2<1<0', array([0, 0, 0, 0, 0, 1])),
 ('5<4<3<2<1<0', array([0, 1, 0, 0, 0, 1])),
 ('5<4<3<2<1<0', array([1, 1, 0, 0, 0, 1]))]

In [ ]:
check_cond_imp_monotone_increasing(Dperms)

[('2<0<4<5<1<3', '2,3,1,5,4,3'),
 ('2<0<4<5<1<3', '2,3,1,5,4,3,0'),
 ('2<0<5<4<1<3', '2,3,1,4,5,3'),
 ('2<0<5<4<1<3', '2,3,1,4,5,3,0'),
 ('2<1<4<5<0<3', '2,3,0,5,4,3'),
 ('2<1<4<5<0<3', '2,3,0,5,4,3,1'),
 ('2<1<5<4<0<3', '2,3,0,4,5,3'),
 ('2<1<5<4<0<3', '2,3,0,4,5,3,1'),
 ('2<4<0<5<1<3', '2,3,1,5,4,3'),
 ('2<4<0<5<1<3', '2,3,1,5,0,4,3'),
 ('2<4<1<5<0<3', '2,3,0,5,4,3'),
 ('2<4<1<5<0<3', '2,3,0,5,1,4,3'),
 ('2<4<5<0<1<3', '2,3,1,5,4,3'),
 ('2<4<5<0<1<3', '2,3,1,0,5,4,3'),
 ('2<4<5<1<0<3', '2,3,0,5,4,3'),
 ('2<4<5<1<0<3', '2,3,0,1,5,4,3'),
 ('2<5<0<4<1<3', '2,3,1,4,5,3'),
 ('2<5<0<4<1<3', '2,3,1,4,0,5,3'),
 ('2<5<1<4<0<3', '2,3,0,4,5,3'),
 ('2<5<1<4<0<3', '2,3,0,4,1,5,3'),
 ('2<5<4<0<1<3', '2,3,1,4,5,3'),
 ('2<5<4<0<1<3', '2,3,1,0,4,5,3'),
 ('2<5<4<1<0<3', '2,3,0,4,5,3'),
 ('2<5<4<1<0<3', '2,3,0,1,4,5,3'),
 ('3<0<1<4<5<2', '3,2,5,1,0,2'),
 ('3<0<1<4<5<2', '3,2,5,4,1,0,2'),
 ('3<0<1<5<4<2', '3,2,4,1,0,2'),
 ('3<0<1<5<4<2', '3,2,4,5,1,0,2'),
 ('3<0<4<1<5<2', '3,2,5,1,0,2'),
 ('3<0<4<1<5<2'

In [ ]:
for i in Dperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
check_h_deltas_repeat(Dperms)

[]

In [ ]:
for i in generate_equivalences(Dperms):
    print(f"{i[0]} = {i[1]}")

0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<2<3<4<0<5 = 1<2<3<0<4<5
1<2<3<4<5<0 = 1<2<3<0<4<5
1<2<3<4<5<0 = 1<2<3<4<0<5
1<2<3<5<0<4 = 1<2<3<0<5<4
1<2<3<5<4<0 = 1<2<3<0<5<4
1<2<3<5<4<0 = 1<2<3<5<0<4
1<2<4<3<5<0 = 1<2<4<3<0<5
1<2<5<3<4<0 = 1<2<5<3<0<4
1<3<2<4<0<5 = 1<3<2<0<4<5
1<3<2<4<5<0 = 1<3<2<0<4<5
1<3<2<4<5<0 = 1<3<2<4<0<5
1<3<2<5<0<4 = 1<3<2<0<5<4
1<3<2<5<4<0 = 1<3<2<0<5<4
1<3<2<5<4<0 = 1<3<2<5<0<4
1<3<4<2<5<0 = 1<3<4<2<0<5
1<3<5<2<4<0 = 1<3<5<2<0<4
1<4<2<3<5<0 = 1<4<2<3<0<5
1<4<3<2<5<0 = 1<4<3<2<0<5
1<5<2<3<4<0 

## Type E

In [74]:
Eperms = gen_permutations('E',6)

In [75]:
for i in check_periodicity_conj_perms(Eperms):
    print(i)

KeyboardInterrupt: 

In [29]:
for i in check_one_smallest_last(Eperms):
    print(i)

In [59]:
for i in check_SL_word_algo_perms(Eperms,3):
    print(i)

KeyboardInterrupt: 

In [ ]:
for i in check_delta_type_prediction_perms(Eperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
check_h_deltas_repeat(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

[]

In [ ]:
for i in Eperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
write_delta_types_to_file(Eperms,"E6DeltaTypes.csv")

  0%|          | 0/5040 [00:00<?, ?it/s]

In [ ]:
for i in generate_equivalences(Eperms):
    print(f"{i[0]} = {i[1]}")

0<1<2<4<5<6<3 = 0<1<2<4<5<3<6
0<1<2<5<4<6<3 = 0<1<2<5<4<3<6
0<1<3<4<5<6<2 = 0<1<3<4<5<2<6
0<1<3<5<4<6<2 = 0<1<3<5<4<2<6
0<1<4<2<5<6<3 = 0<1<4<2<5<3<6
0<1<4<3<5<6<2 = 0<1<4<3<5<2<6
0<1<4<5<2<6<3 = 0<1<4<5<2<3<6
0<1<4<5<3<6<2 = 0<1<4<5<3<2<6
0<1<4<5<6<2<3 = 0<1<4<5<2<3<6
0<1<4<5<6<2<3 = 0<1<4<5<2<6<3
0<1<4<5<6<3<2 = 0<1<4<5<3<2<6
0<1<4<5<6<3<2 = 0<1<4<5<3<6<2
0<1<5<2<4<6<3 = 0<1<5<2<4<3<6
0<1<5<3<4<6<2 = 0<1<5<3<4<2<6
0<1<5<4<2<6<3 = 0<1<5<4<2<3<6
0<1<5<4<3<6<2 = 0<1<5<4<3<2<6
0<1<5<4<6<2<3 = 0<1<5<4<2<3<6
0<1<5<4<6<2<3 = 0<1<5<4<2<6<3
0<1<5<4<6<3<2 = 0<1<5<4<3<2<6
0<1<5<4<6<3<2 = 0<1<5<4<3<6<2
0<2<1<4<5<6<3 = 0<2<1<4<5<3<6
0<2<1<5<4<6<3 = 0<2<1<5<4<3<6
0<2<3<4<5<1<6 = 0<2<3<4<1<5<6
0<2<3<4<5<6<1 = 0<2<3<4<1<5<6
0<2<3<4<5<6<1 = 0<2<3<4<5<1<6
0<2<3<4<6<1<5 = 0<2<3<4<1<6<5
0<2<3<4<6<5<1 = 0<2<3<4<1<6<5
0<2<3<4<6<5<1 = 0<2<3<4<6<1<5
0<2<3<5<4<6<1 = 0<2<3<5<4<1<6
0<2<3<5<6<1<4 = 0<2<3<5<1<6<4
0<2<3<6<4<5<1 = 0<2<3<6<4<1<5
0<2<4<1<5<6<3 = 0<2<4<1<5<3<6
0<2<4<3<5<1<6 = 0<2<4<3<1<5<6
0<2<4<3<5<

KeyboardInterrupt: 

In [ ]:
get_max_periodicity(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

(10, '2<0<1<4<6<5<3', array([1, 0, 1, 1, 1, 1, 0]))

## Type F

In [646]:
Fperms = gen_permutations('F',4)

In [23]:
for i in check_periodicity_conj_perms(Fperms):
    print(i)

In [600]:
F4 = rootSystem([1,0,2,3,4],'F')
w = [2,3,5,6,2]
print(F4.get_words(w)[0])
print(F4.parse_to_delta_format(F4.get_words(w)[0]))
print(F4.SL_word_algo(w))
print(F4.parse_to_delta_format(F4.SL_word_algo(w)))

1,2,3,3,2,0,1,2,3,4,3,0,1,2,3,4,3,2
['123320', [1, 1]]
1,2,3,3,2,0,1,2,3,4,3,2,3,0,1,2,3,4
['123320123432301234']


In [239]:
print(rootSystem([1,0,2,3,4],'F').SL_word_algo([2,3,5,6,2]).noCommas())

123320123431234320


In [647]:
for i in check_SL_word_algo_perms(Fperms,3):
    print(i)

('1<0<2<3<4', array([2, 5, 7, 8, 4]), ['123320', [1, 1], '12341234'], ['123320', [2, 1], '12341234'])
('1<0<2<3<4', array([2, 5, 7, 9, 4]), ['123320', [1, 1], '123431234'], ['123320', [2, 1], '123431234'])
('1<0<2<3<4', array([ 2,  5,  7, 10,  4]), ['123320', [1, 1], '1234312343'], ['123320', [2, 1], '1234312343'])
('1<0<2<3<4', array([ 3,  6,  8, 10,  5]), ['123320', [1, 1], '12343123401234'], ['123320', [2, 1], '12343123401234'])
('1<0<2<3<4', array([ 3,  6,  8, 11,  5]), ['123320', [1, 1], '123431234301234'], ['123320', [2, 1], '123431234301234'])
('1<0<2<4<3', array([ 2,  5,  7, 10,  4]), ['123320', [1, 1], '1233412334'], ['123320', [2, 1], '1233412334'])
('1<0<3<2<4', array([2, 5, 7, 8, 4]), ['123320', [1, 1], '12341234'], ['123320', [2, 1], '12341234'])
('1<0<3<2<4', array([2, 5, 7, 9, 4]), ['123320', [1, 1], '123431234'], ['123320', [2, 1], '123431234'])
('1<0<3<2<4', array([ 2,  5,  7, 10,  4]), ['123320', [1, 1], '1234312343'], ['123320', [2, 1], '1234312343'])
('1<0<3<2<4', a

KeyboardInterrupt: 

In [ ]:
for i in check_delta_type_prediction_perms(Fperms):
    print(i)

  0%|          | 0/120 [00:00<?, ?it/s]

('F', '2<0<1<4<3', 2, 'lastSmallest', 'cofac')
('F', '2<0<4<1<3', 2, 'lastSmallest', 'cofac')
('F', '2<1<0<4<3', 2, 'lastSmallest', 'cofac')
('F', '2<1<4<0<3', 2, 'lastSmallest', 'cofac')
('F', '2<1<4<3<0', 2, 'lastSmallest', 'cofac')
('F', '2<4<0<1<3', 3, 'lastSmallest', 'cofac')
('F', '2<4<1<0<3', 3, 'lastSmallest', 'cofac')
('F', '2<4<1<3<0', 3, 'lastSmallest', 'cofac')
('F', '3<0<1<2<4', 2, 'lastSmallest', 'cofac')
('F', '3<0<2<1<4', 2, 'lastSmallest', 'cofac')
('F', '3<0<2<4<1', 2, 'lastSmallest', 'cofac')
('F', '3<1<0<2<4', 2, 'lastSmallest', 'cofac')
('F', '3<1<2<0<4', 2, 'lastSmallest', 'cofac')
('F', '3<1<2<4<0', 2, 'lastSmallest', 'cofac')
('F', '3<2<0<1<4', 2, 'lastSmallest', 'cofac')
('F', '3<2<1<0<4', 2, 'lastSmallest', 'cofac')
('F', '3<2<0<4<1', 2, 'lastSmallest', 'cofac')
('F', '3<2<1<4<0', 2, 'lastSmallest', 'cofac')
('F', '3<2<4<1<0', 2, 'lastSmallest', 'cofac')
('F', '3<2<4<0<1', 2, 'lastSmallest', 'cofac')


In [ ]:
write_delta_types_to_file(Fperms,"FDeltaTypes.csv",notOnlyAllLast=False)

  0%|          | 0/120 [00:00<?, ?it/s]

In [ ]:
check_monotone_increasing_imp_cond(Fperms)

UnboundLocalError: cannot access local variable 'returnArr' where it is not associated with a value

In [ ]:
check_cond_imp_monotone_increasing(Fperms)

[('1<0<2<3<4', '1,2,3,4,3,4,2'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,2'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,2,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,2'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,2,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,2'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,2,0'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4,3'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4,0'),
 ('1<0<3<

In [ ]:
for i in Fperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
check_h_deltas_repeat(Fperms)

[]

In [ ]:
get_max_periodicity(Fperms)

(11, '1<0<3<4<2', array([0, 1, 2, 2, 0]))

In [ ]:
for i in generate_equivalences(Fperms):
    print(f"{i[0]} = {i[1]}")

0<2<3<4<1 = 0<2<3<1<4
0<3<2<4<1 = 0<3<2<1<4
1<2<3<0<4 = 1<2<0<3<4
1<2<3<4<0 = 1<2<0<3<4
1<2<3<4<0 = 1<2<3<0<4
1<2<4<0<3 = 1<2<0<4<3
1<2<4<3<0 = 1<2<0<4<3
1<2<4<3<0 = 1<2<4<0<3
1<3<2<4<0 = 1<3<2<0<4
1<3<4<0<2 = 1<3<0<4<2
1<4<2<3<0 = 1<4<2<0<3
2<0<3<4<1 = 2<0<3<1<4
2<1<3<0<4 = 2<1<0<3<4
2<1<3<4<0 = 2<1<0<3<4
2<1<3<4<0 = 2<1<3<0<4
2<1<4<0<3 = 2<1<0<4<3
2<1<4<3<0 = 2<1<0<4<3
2<1<4<3<0 = 2<1<4<0<3
2<3<0<4<1 = 2<3<0<1<4
2<3<1<4<0 = 2<3<1<0<4
2<3<4<0<1 = 2<3<0<1<4
2<3<4<0<1 = 2<3<0<4<1
2<3<4<1<0 = 2<3<1<0<4
2<3<4<1<0 = 2<3<1<4<0
2<4<1<3<0 = 2<4<1<0<3
3<0<2<4<1 = 3<0<2<1<4
3<1<2<4<0 = 3<1<2<0<4
3<1<4<0<2 = 3<1<0<4<2
3<2<0<4<1 = 3<2<0<1<4
3<2<1<4<0 = 3<2<1<0<4
3<2<4<0<1 = 3<2<0<1<4
3<2<4<0<1 = 3<2<0<4<1
3<2<4<1<0 = 3<2<1<0<4
3<2<4<1<0 = 3<2<1<4<0
4<1<2<3<0 = 4<1<2<0<3
4<2<1<3<0 = 4<2<1<0<3
